In [ ]:
!pip install segmentation-models-pytorch albumentations timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.8 MB/s eta 0:00:00


In [4]:
# ==========================================
# 1. SETUP & INSTALLATION
# ==========================================
!pip install -q segmentation-models-pytorch albumentations

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import glob

# ==========================================
# 2. CONFIGURATION
# ==========================================
CONFIG = {
    "ROOT_DIR": "/content/drive/MyDrive/Colab Notebooks/unzipped_files",
    # Path to save checkpoints so they survive a crash
    "CHECKPOINT_PATH": "/content/drive/MyDrive/Colab Notebooks/unzipped_files/checkpoint.pth",
    "BEST_MODEL_PATH": "/content/drive/MyDrive/Colab Notebooks/unzipped_files/best_model_unetpp.pth",
    "IMG_SIZE": 512,
    "BATCH_SIZE": 8,
    "EPOCHS": 40,
    "LR": 1e-4,
    "ENCODER": "efficientnet-b4",
    "WEIGHTS": "imagenet",
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "NUM_CLASSES": 10
}

print(f"✅ Running on: {CONFIG['DEVICE']}")

ID_TO_INDEX = {
    100: 0, 200: 1, 300: 2, 500: 3, 550: 4,
    600: 5, 700: 6, 800: 7, 7100: 8, 10000: 9
}

# ==========================================
# 3. DATASET LOADER
# ==========================================
class OffroadDataset(Dataset):
    def __init__(self, root_dir, subset="train", transform=None):
        self.transform = transform
        self.subset = subset

        if subset == "test_public_80":
             self.base_path = os.path.join(root_dir, "test_public_80")
        else:
             self.base_path = os.path.join(root_dir, "Offroad_Segmentation_Training_Dataset", subset)

        self.img_dir = os.path.join(self.base_path, "Color_Images")
        self.mask_dir = os.path.join(self.base_path, "Segmentation")

        self.images = sorted(glob.glob(os.path.join(self.img_dir, "*.jpg")) +
                             glob.glob(os.path.join(self.img_dir, "*.png")))

    def __len__(self):
        return len(self.images)

    def map_mask(self, mask):
        new_mask = np.zeros_like(mask, dtype=np.uint8)
        for id_val, idx_val in ID_TO_INDEX.items():
            new_mask[mask == id_val] = idx_val
        return new_mask

    def __getitem__(self, idx):
        img_path = self.images[idx]
        filename = os.path.basename(img_path)
        mask_filename = os.path.splitext(filename)[0] + ".png"
        mask_path = os.path.join(self.mask_dir, mask_filename)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, -1)

        if mask is None:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        else:
            mask = self.map_mask(mask)
            mask = mask.astype(np.uint8)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# ==========================================
# 4. AUGMENTATIONS
# ==========================================
def get_transforms(phase="train"):
    if phase == "train":
        return A.Compose([
            A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(scale_limit=0.2, rotate_limit=30, shift_limit=0.1, p=0.7),
            A.OneOf([A.ISONoise(p=1), A.GaussNoise(var_limit=(10, 50), p=1)], p=0.5),
            A.OneOf([A.RandomBrightnessContrast(p=1), A.HueSaturationValue(p=1), A.RGBShift(p=1)], p=0.5),
            A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
            A.Normalize(),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
            A.Normalize(),
            ToTensorV2()
        ])

# ==========================================
# 5. RESUMABLE TRAINING LOOP
# ==========================================
def train_model():
    train_ds = OffroadDataset(CONFIG['ROOT_DIR'], subset="train", transform=get_transforms("train"))
    val_ds = OffroadDataset(CONFIG['ROOT_DIR'], subset="val", transform=get_transforms("val"))

    train_loader = DataLoader(train_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

    model = smp.UnetPlusPlus(
        encoder_name=CONFIG['ENCODER'],
        encoder_weights=CONFIG['WEIGHTS'],
        in_channels=3,
        classes=CONFIG['NUM_CLASSES'],
    ).to(CONFIG['DEVICE'])

    loss_ce = nn.CrossEntropyLoss()
    loss_dice = smp.losses.DiceLoss(mode='multiclass', from_logits=True)

    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['LR'], weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    scaler = GradScaler()

    best_loss = float("inf")
    start_epoch = 0

    # 🔄 CHECKPOINT LOADING LOGIC
    if os.path.exists(CONFIG['CHECKPOINT_PATH']):
        print(f"🔄 Checkpoint found! Resuming training...")
        checkpoint = torch.load(CONFIG['CHECKPOINT_PATH'])
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_loss = checkpoint['best_loss']
        print(f"⏩ Resuming from Epoch {start_epoch+1}")
    else:
        print("🚀 Starting fresh training...")

    for epoch in range(start_epoch, CONFIG['EPOCHS']):
        model.train()
        train_loss = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in loop:
            imgs, masks = imgs.to(CONFIG['DEVICE']), masks.to(CONFIG['DEVICE'])

            with autocast():
                logits = model(imgs)
                l1 = loss_ce(logits, masks)
                l2 = loss_dice(logits, masks)
                loss = l1 * 0.4 + l2 * 0.6

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(CONFIG['DEVICE']), masks.to(CONFIG['DEVICE'])
                with autocast():
                    logits = model(imgs)
                    l1 = loss_ce(logits, masks)
                    l2 = loss_dice(logits, masks)
                    val_loss += (l1 * 0.4 + l2 * 0.6).item()

        avg_val_loss = val_loss / len(val_loader)
        scheduler.step(avg_val_loss)

        print(f"📉 Val Loss: {avg_val_loss:.4f}")

        # 💾 SAVE CHECKPOINT (EVERY EPOCH)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_loss,
        }, CONFIG['CHECKPOINT_PATH'])

        # 💾 SAVE BEST MODEL
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), CONFIG['BEST_MODEL_PATH'])
            print("🏆 Best Model Saved!")

    return model

# ==========================================
# 6. RUN
# ==========================================
model = train_model()

✅ Running on: cuda


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-3925743426.py:106: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.OneOf([A.ISONoise(p=1), A.GaussNoise(var_limit=(10, 50), p=1)], p=0.5),
/tmp/ipython-input-3925743426.py:108: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
/tmp/ipython-input-3925743426.py:141: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


🔄 Checkpoint found! Resuming training...
⏩ Resuming from Epoch 29


Epoch 29:   0%|          | 0/358 [00:00<?, ?it/s]

/tmp/ipython-input-3925743426.py:166: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipython-input-3925743426.py:186: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


...

📉 Val Loss: 0.2832
🏆 Best Model Saved!


Epoch 39:   0%|          | 0/358 [00:00<?, ?it/s]

📉 Val Loss: 0.2832


Epoch 40:   0%|          | 0/358 [00:00<?, ?it/s]

📉 Val Loss: 0.2833


In [5]:
# ==========================================
# 📊 AUTOMATED COMPETITION GRADER
# ==========================================
import pandas as pd
import time

# 1. Define the Official Metric (IoU)
def compute_iou_batch(preds, labels, num_classes=10):
    # Flatten: (Batch, Height, Width) -> (Batch * Height * Width)
    preds = preds.view(-1)
    labels = labels.view(-1)

    iou_per_class = []
    # Ignore classes that are not present in the ground truth to avoid NaN
    present_classes = torch.unique(labels)

    for cls in range(num_classes):
        pred_inds = preds == cls
        target_inds = labels == cls

        intersection = (pred_inds & target_inds).sum().float()
        union = (pred_inds | target_inds).sum().float()

        if union == 0:
            # If class is missing from both, strictly it's NaN,
            # but usually ignored in mIoU calculation
            iou_per_class.append(float('nan'))
        else:
            iou_per_class.append((intersection / union).item())

    return iou_per_class

# 2. The Evaluation Loop
def run_official_evaluation():
    print("🚀 Starting Official Grading Protocol...")

    # Load Valid Set (This is the only data with Ground Truths we can test on)
    val_ds = OffroadDataset(CONFIG['ROOT_DIR'], subset="val", transform=get_transforms("val"))
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

    # Load Your Best Model
    model = smp.UnetPlusPlus(
        encoder_name=CONFIG['ENCODER'], in_channels=3, classes=CONFIG['NUM_CLASSES']
    ).to(CONFIG['DEVICE'])

    if os.path.exists(CONFIG['BEST_MODEL_PATH']):
        model.load_state_dict(torch.load(CONFIG['BEST_MODEL_PATH']))
        print(f"✅ Loaded weights from: {CONFIG['BEST_MODEL_PATH']}")
    else:
        print("❌ Error: No model found! Train first.")
        return

    model.eval()

    # Metrics Storage
    class_ious = {i: [] for i in range(CONFIG['NUM_CLASSES'])}
    inference_times = []

    print(f"📂 Evaluating {len(val_ds)} validation images...")

    with torch.no_grad():
        for i, (image, mask) in enumerate(tqdm(val_loader)):
            image = image.to(CONFIG['DEVICE'])
            mask = mask.to(CONFIG['DEVICE'])

            # Speed Test ⚡
            start = time.time()
            logits = model(image)
            pred_mask = torch.argmax(logits, dim=1)
            end = time.time()
            inference_times.append((end - start) * 1000) # ms

            # IoU Calculation 🧮
            batch_ious = compute_iou_batch(pred_mask, mask, CONFIG['NUM_CLASSES'])

            for cls_idx, iou in enumerate(batch_ious):
                if not np.isnan(iou):
                    class_ious[cls_idx].append(iou)

    # 3. Generate Report
    print("\n" + "="*50)
    print("🏆 FINAL COMPETITION SCORECARD")
    print("="*50)

    # Speed
    avg_speed = np.mean(inference_times)
    print(f"⚡ Avg Inference Speed: {avg_speed:.2f} ms")
    if avg_speed < 50: print("   ✅ PASS (< 50ms)")
    else: print("   ⚠️ FAIL (> 50ms) - Try reducing IMG_SIZE if strictly enforced")

    # IoU Report
    class_names = ["Trees", "Lush Bushes", "Dry Grass", "Dry Bushes", "Ground Clutter",
                   "Flowers", "Logs", "Rocks", "Landscape", "Sky"]

    final_class_scores = []
    for i in range(CONFIG['NUM_CLASSES']):
        mean_score = np.mean(class_ious[i]) if class_ious[i] else 0.0
        final_class_scores.append(mean_score)

    mIoU = np.mean(final_class_scores)

    # Create DataFrame for nice display
    df = pd.DataFrame({
        "Class Name": class_names,
        "IoU Score": [f"{s:.4f}" for s in final_class_scores]
    })

    print(f"\n📊 Mean IoU (mIoU): {mIoU:.4f}")
    print("-" * 30)
    print(df.to_string(index=False))
    print("="*50)

    return mIoU, df

# RUN THE GRADER
mIoU, results = run_official_evaluation()

🚀 Starting Official Grading Protocol...
✅ Loaded weights from: /content/drive/MyDrive/Colab Notebooks/unzipped_files/best_model_unetpp.pth
📂 Evaluating 317 validation images...


  0%|          | 0/317 [00:00<?, ?it/s]


🏆 FINAL COMPETITION SCORECARD
⚡ Avg Inference Speed: 42.10 ms
   ✅ PASS (< 50ms)

📊 Mean IoU (mIoU): 0.5705
------------------------------
    Class Name IoU Score
         Trees    0.6747
   Lush Bushes    0.5674
     Dry Grass    0.6567
    Dry Bushes    0.4797
Ground Clutter    0.3679
       Flowers    0.6190
          Logs    0.3247
         Rocks    0.3824
     Landscape    0.6460
           Sky    0.9861


In [6]:
# ==========================================
# 🧪 FINAL TEST SET EVALUATION (test_public_80)
# ==========================================
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
import glob
import pandas as pd
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
import time

# --- CONFIGURATION ---
TEST_CONFIG = {
    # PATH TO YOUR TEST SET
    "TEST_DIR": "/content/drive/MyDrive/Colab Notebooks/unzipped_files/test_public_80",
    "MODEL_PATH": "/content/drive/MyDrive/Colab Notebooks/unzipped_files/best_model_unetpp.pth",
    "OUTPUT_DIR": "./test_results_public_80",
    "IMG_SIZE": 512,
    "ENCODER": "efficientnet-b4",
    "NUM_CLASSES": 10,
    "DEVICE": torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Visualization Colors (BGR)
COLOR_MAP = {
    0: (0, 0, 0),       1: (34, 139, 34),   2: (0, 255, 0),     3: (140, 180, 210),
    4: (43, 90, 139),   5: (0, 128, 128),   6: (19, 69, 139),   7: (128, 128, 128),
    8: (45, 82, 160),   9: (235, 206, 135)
}
CLASS_NAMES = ['Background', 'Trees', 'Lush Bushes', 'Dry Grass', 'Dry Bushes',
               'Ground Clutter', 'Logs', 'Rocks', 'Landscape', 'Sky']

ID_TO_INDEX = {
    100: 0, 200: 1, 300: 2, 500: 3, 550: 4,
    600: 5, 700: 6, 800: 7, 7100: 8, 10000: 9
}

# --- DATASET LOADER ---
class TestDataset(Dataset):
    def __init__(self, root_dir):
        self.image_dir = os.path.join(root_dir, 'Color_Images')
        # Handle case sensitivity
        p1 = os.path.join(root_dir, 'segmentation')
        p2 = os.path.join(root_dir, 'Segmentation')
        self.masks_dir = p2 if os.path.exists(p2) else p1

        self.images = sorted(glob.glob(os.path.join(self.image_dir, "*.*")))
        self.transform = A.Compose([
            A.Resize(TEST_CONFIG['IMG_SIZE'], TEST_CONFIG['IMG_SIZE']),
            A.Normalize(),
            ToTensorV2()
        ])

        # Verify
        if len(self.images) == 0:
            print(f"❌ CRITICAL ERROR: No images found in {self.image_dir}")

    def __len__(self): return len(self.images)

    def map_mask(self, mask):
        new_mask = np.zeros_like(mask, dtype=np.uint8)
        for k, v in ID_TO_INDEX.items(): new_mask[mask == k] = v
        return new_mask

    def __getitem__(self, idx):
        img_path = self.images[idx]
        filename = os.path.basename(img_path)
        mask_path = os.path.join(self.masks_dir, os.path.splitext(filename)[0] + ".png")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Try loading mask
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, -1)
            mask = self.map_mask(mask)
        else:
            # If no mask (blind test), return zeros
            mask = np.zeros(image.shape[:2], dtype=np.uint8)

        aug = self.transform(image=image, mask=mask)
        return aug['image'], aug['mask'].long(), image, filename

# --- METRIC CALCULATION ---
def compute_iou(pred, target, num_classes=10):
    pred = torch.argmax(pred, dim=1).view(-1)
    target = target.view(-1)
    iou_list = []
    # Ignore background class 0 if needed, but usually included
    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float()
        union = (pred_inds | target_inds).sum().float()

        if union == 0:
            iou_list.append(float('nan')) # Class not present in this image
        else:
            iou_list.append((intersection / union).item())
    return np.nanmean(iou_list), iou_list

# --- MAIN EXECUTION ---
def run_test_evaluation():
    print(f"🚀 Loading Model from: {TEST_CONFIG['MODEL_PATH']}")
    if not os.path.exists(TEST_CONFIG['MODEL_PATH']):
        print("❌ Error: Model file missing.")
        return

    # Load Model
    model = smp.UnetPlusPlus(
        encoder_name=TEST_CONFIG['ENCODER'], in_channels=3, classes=TEST_CONFIG['NUM_CLASSES']
    ).to(TEST_CONFIG['DEVICE'])
    model.load_state_dict(torch.load(TEST_CONFIG['MODEL_PATH'], map_location=TEST_CONFIG['DEVICE']))
    model.eval()

    # Load Data
    ds = TestDataset(TEST_CONFIG['TEST_DIR'])
    loader = DataLoader(ds, batch_size=1, shuffle=False)
    print(f"📂 Found {len(ds)} Test Images in 'test_public_80'")

    os.makedirs(TEST_CONFIG['OUTPUT_DIR'], exist_ok=True)
    viz_dir = os.path.join(TEST_CONFIG['OUTPUT_DIR'], "visualizations")
    os.makedirs(viz_dir, exist_ok=True)

    inference_times = []
    all_ious = []
    class_ious = {i: [] for i in range(TEST_CONFIG['NUM_CLASSES'])}

    print("⚡ Running Inference on Test Set...")
    with torch.no_grad():
        for i, (img_t, mask_t, original_img, fname) in enumerate(tqdm(loader)):
            img_t, mask_t = img_t.to(TEST_CONFIG['DEVICE']), mask_t.to(TEST_CONFIG['DEVICE'])

            # 1. Inference Speed
            start = time.time()
            logits = model(img_t)
            end = time.time()
            inference_times.append((end - start) * 1000)

            # 2. Metrics
            # Note: We only calculate IoU if the mask is not all zeros (valid ground truth)
            if mask_t.max() > 0 or mask_t.sum() > 0:
                mean_iou, cls_iou_list = compute_iou(logits, mask_t, TEST_CONFIG['NUM_CLASSES'])
                all_ious.append(mean_iou)
                for cls_idx, val in enumerate(cls_iou_list):
                    if not np.isnan(val): class_ious[cls_idx].append(val)

            # 3. Visualizations (Save first 10 + any with specific filename if you want)
            if i < 10:
                pred_mask = torch.argmax(logits, dim=1).squeeze().cpu().numpy().astype(np.uint8)

                # Create Color Mask
                h, w = pred_mask.shape
                viz_pred = np.zeros((h, w, 3), dtype=np.uint8)
                for k, v in COLOR_MAP.items(): viz_pred[pred_mask == k] = v

                # Resize original to 512 for stacking
                orig_rez = cv2.resize(original_img[0].numpy(), (512, 512))

                # Stack: Original | Prediction
                viz_final = np.hstack([orig_rez, viz_pred])

                save_path = os.path.join(viz_dir, f"pred_{fname[0]}")
                cv2.imwrite(save_path, cv2.cvtColor(viz_final, cv2.COLOR_RGB2BGR))

    # --- REPORT GENERATION ---
    avg_speed = np.mean(inference_times)
    final_mIoU = np.nanmean(all_ious) if all_ious else 0.0

    print("\n" + "="*40)
    print(f"🏆 TEST SET RESULTS (test_public_80)")
    print("="*40)
    print(f"✅ Mean IoU (mIoU):  {final_mIoU:.4f}")
    print(f"⚡ Avg Inference:    {avg_speed:.2f} ms")

    # Per-Class Table
    res_data = []
    for i in range(TEST_CONFIG['NUM_CLASSES']):
        score = np.mean(class_ious[i]) if class_ious[i] else 0.0
        res_data.append({"Class": CLASS_NAMES[i], "IoU": score})

    df = pd.DataFrame(res_data)
    print("\n📊 Per-Class Breakdown:")
    print(df.to_string(index=False))

    # Save Report
    with open(f"{TEST_CONFIG['OUTPUT_DIR']}/final_test_report.txt", "w") as f:
        f.write(f"Test Set Evaluation\n")
        f.write(f"Mean IoU: {final_mIoU:.4f}\n")
        f.write(f"Speed: {avg_speed:.2f} ms\n\n")
        f.write(df.to_string())

    print(f"\n📁 Results saved to: {TEST_CONFIG['OUTPUT_DIR']}")

# Run It
run_test_evaluation()

🚀 Loading Model from: /content/drive/MyDrive/Colab Notebooks/unzipped_files/best_model_unetpp.pth
📂 Found 801 Test Images in 'test_public_80'
⚡ Running Inference on Test Set...


  0%|          | 0/801 [00:00<?, ?it/s]


🏆 TEST SET RESULTS (test_public_80)
✅ Mean IoU (mIoU):  0.3431
⚡ Avg Inference:    25.13 ms

📊 Per-Class Breakdown:
         Class      IoU
    Background 0.407076
         Trees 0.000599
   Lush Bushes 0.473537
     Dry Grass 0.276398
    Dry Bushes 0.000000
Ground Clutter 0.000000
          Logs 0.000000
         Rocks 0.021010
     Landscape 0.671417
           Sky 0.976393

📁 Results saved to: ./test_results_public_80
